In [9]:
import numpy as np
import random
import torch
from models.gconvGRU import GConvGRUModel
from memory_capacity_utils import gen_lag_data, get_mem_cap_from_model

In [10]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print('running on GPU')
else:
    device = torch.device("cpu")
    print('running on CPU')

running on GPU


In [13]:
dataset = np.load("datasets/multivariate_simulation_data_2.npy")
dataset = torch.from_numpy(dataset).squeeze()
dataset = dataset.type(torch.FloatTensor)
dataset = dataset[:, :, :, :]

In [14]:
manual_seed = 777
max_lag = 35
np.random.seed(manual_seed)
random.seed(manual_seed)
torch.manual_seed(manual_seed)

# Reservoir signals
X_train_res_np, y_train_res_np = gen_lag_data(1000, 41, max_lag)
X_test_res_np, y_test_res_np = gen_lag_data(500, 42, max_lag)
X_train_res = torch.from_numpy(X_train_res_np).unsqueeze(1).to(device, dtype=torch.float64)
X_test_res = torch.from_numpy(X_test_res_np).unsqueeze(1).to(device, dtype=torch.float64)
y_train_res = torch.from_numpy(y_train_res_np).to(device, dtype=torch.float64)
y_test_res = torch.from_numpy(y_test_res_np).to(device, dtype=torch.float64)

In [18]:
torch.manual_seed(1)
input_weights = (torch.rand((35, 1), dtype=torch.float64) * 2.0 - 1.0).to(device)
model = GConvGRUModel(device=device, input_weights=input_weights, input_scaling=1e-6).to(device)
mem_caps = np.zeros((dataset.shape[0], dataset.shape[1]))
for n_subjects in range(dataset.shape[0]):
    for n_t in range(dataset.shape[1]):
        mem_caps[n_subjects, n_t] = get_mem_cap_from_model(model, dataset[n_subjects, n_t, :, :].to(device), X_train_res, y_train_res, X_test_res, y_test_res).item()

In [ ]:
np.save('mem_caps_2', mem_caps)